In [ ]:
!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq
from pandas_datareader import data as pdr
yf.pdr_override() # <== thats all it takes :-)
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt

     |████████████████████████████████| 6.3 MB 6.3 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
nse_Symbol=['ADANIPORTS.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJAJFINSV.NS',
 'BAJFINANCE.NS',
 'BHARTIARTL.NS',
 'BPCL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DIVISLAB.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFC.NS',
 'HDFCBANK.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'IOC.NS',
 'ITC.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBILIFE.NS',
 'SBIN.NS',
 'SHREECEM.NS',
 'SUNPHARMA.NS',
 'TATACONSUM.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TCS.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'UPL.NS',
 'WIPRO.NS']

nifty_only=['^NSEI']

In [ ]:
def checker1(stock_name,start_date,end_date):
  df1 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date)
  # df1 = yf.download('SBIN.NS', 
  #                   start='2021-10-01', 
  #                   end='2021-11-01')
  value=[]
  for i in range(0,len(df1)):
    if i==0:
      value.append('None')
      continue
    if df1['Open'][i] > df1['High'][i-1]:
      value.append('PDH')
    elif df1['Open'][i] < df1['Low'][i-1]:
      value.append('PDL')
    else:
      value.append('None')
  df1['Cond1']=value
  df1=df1[df1['Cond1'] != 'None']
  df1=df1.reset_index()
  df1['Date']=df1['Date'].astype(str)
  df1=df1[['Date','Cond1']]

  df2 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    interval='5m')

  # df2 = yf.download('SBIN.NS', 
  #                   start='2021-10-01', 
  #                   end='2021-11-01',
  #                   interval='5m')
  df2.reset_index(inplace=True)
  df2['Date']=df2['Datetime'].astype(str).str.split(' ',expand=True)[0]
  df2['Time']=df2['Datetime'].astype(str).str.split(' ',expand=True)[1]
  df2=df2[df2['Time'].str.contains('09:15')]
  df2=df2.reset_index().drop('index',axis=1)
  df2['Cond2']=['OH' if df2['Open'][i] == df2['High'][i] else 'OL' if df2['Open'][i] == df2['Low'][i] else 'None' for i in range(0,len(df2))]
  df2=df2[df2['Cond2'] != 'None']
  df2=df2.reset_index().drop('index',axis=1)
  df2=df2[['Date','Cond2']]
  df3=pd.merge(df1,df2,on='Date',how='inner')
  return (list(df3['Cond2']),list(df3['Date']))

In [ ]:
dold='2021-10-01'
dtod='2021-11-27'

In [ ]:
# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(nifty_only):
    selected_stocks[i]=checker1(i,dold,dtod)
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Cond1','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
# print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')

  0%|          | 0/1 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
final['Date']

0    2021-10-13
1    2021-10-19
2    2021-11-02
3    2021-11-08
4    2021-11-10
5    2021-11-15
Name: Date, dtype: object

In [ ]:
# final.to_csv('nov_15days_pdh_op_low.csv',index=False)

In [ ]:
files.download('nov_15days_pdh_op_low.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
rm 'nov_15days_pdh_op_low.csv'

# CONTINUE to only PDH PDL

In [ ]:
# n
# stocks_list=['EXIDEIND.NS','TATAMOTORS.NS','MARUTI.NS',
#              'KOTAKBANK.NS','SBIN.NS','ICICIBANK.NS',
#              'WIPRO.NS','HCLTECH.NS','TECHM.NS',
#              'DRREDDY.NS','BIOCON.NS','SUNPHARMA.NS',
#              'JSWSTEEL.NS','JINDALSTEL.NS','HINDALCO.NS',
#              'RELIANCE.NS','TATAPOWER.NS','BPCL.NS',
#              'DLF.NS','GODREJPROP.NS','PRESTIGE.NS',
#              'TATACONSUM.NS','DABUR.NS','HINDUNILVR.NS']

new_list=['MARUTI.NS','HDFCBANK.NS','BAJFINANCE.NS',
          'HINDUNILVR.NS','INFY.NS','ZEEL.NS','TATASTEEL.NS',
          'SUNPHARMA.NS','SBIN.NS','GODREJPROP.NS']

In [ ]:
def checker2(stock_name,start_date,end_date):
  df1 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date)
  # df1 = yf.download('SBIN.NS', 
  #                   start='2021-10-01', 
  #                   end='2021-11-01')
  value=[]
  for i in range(0,len(df1)):
    if i==0:
      value.append('None')
      continue
    if df1['Open'][i] > df1['High'][i-1]:
      value.append('PDH')
    elif df1['Open'][i] < df1['Low'][i-1]:
      value.append('PDL')
    else:
      value.append('None')
  df1['Cond1']=value
  df1=df1[df1['Cond1'] != 'None']
  df1=df1.reset_index()
  df1['Date']=df1['Date'].astype(str)
  df1=df1[['Date','Cond1']]
  return (list(df1['Cond1']),list(df1['Date']))

In [ ]:
dold='2021-09-01'
dtod='2021-10-01'

# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(new_list):
    selected_stocks[i]=checker2(i,dold,dtod)
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Cond1','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
# print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')
final['Date'].value_counts()

  0%|          | 0/10 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

2021-09-14    6
2021-09-17    5
2021-09-24    3
2021-09-22    3
2021-09-30    2
2021-09-23    2
2021-09-27    2
2021-09-29    2
2021-09-20    2
2021-09-08    2
2021-09-02    2
2021-09-06    1
2021-09-21    1
2021-09-03    1
2021-09-28    1
2021-09-16    1
2021-09-15    1
2021-09-13    1
2021-09-09    1
Name: Date, dtype: int64

In [ ]:
final

,Stock,Date
0,MARUTI.NS,2021-09-02
1,MARUTI.NS,2021-09-08
2,MARUTI.NS,2021-09-09
3,MARUTI.NS,2021-09-13
4,MARUTI.NS,2021-09-14
5,MARUTI.NS,2021-09-17
6,MARUTI.NS,2021-09-23
7,HDFCBANK.NS,2021-09-02
8,HDFCBANK.NS,2021-09-15
9,HDFCBANK.NS,2021-09-17


In [ ]:
final.to_excel('pdh_pdl_sep.xlsx',index=False)

In [ ]:
files.download('pdh_pdl_sep.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
rm 'pdh_pdl_sep.xlsx'